In [51]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from sklearn.metrics import classification_report
from scipy.stats import mode
from imblearn.over_sampling import SMOTE
import random

RANDOM_SEED = 12345

In [2]:
#REMOVE THIS LATER, ONLY USED FOR TESTING
train_data = pd.read_csv("Dataset/train.csv")

X = train_data.drop('target', axis=1)
y = train_data.target

X.drop('id', axis = 1, inplace=True)

In [47]:
#TESTING OVERSAMPLING TECHNIQUE
X_train,X_test,y_train,y_test = train_test_split(X,y, test_size=0.3)
ros = RandomOverSampler()
x_sample, y_sample = ros.fit_resample(X_train, y_train)
clf = DecisionTreeClassifier()
clf.fit(x_sample,y_sample)
values = clf.predict(X_test)
print(classification_report(values, y_test))
print((y_sample == 0).sum())
print((y_sample == 1).sum())

              precision    recall  f1-score   support

           0       0.96      0.96      0.96    171509
           1       0.05      0.05      0.05      7055

    accuracy                           0.93    178564
   macro avg       0.50      0.50      0.50    178564
weighted avg       0.92      0.93      0.93    178564



In [ ]:
#TESTING UNDERSAMPLING TECHNIQUE
X_train,X_test,y_train,y_test = train_test_split(X,y, test_size=0.3)
rus = RandomUnderSampler()
x_sample, y_sample = rus.fit_resample(X_train, y_train)
clf = DecisionTreeClassifier()
clf.fit(x_sample,y_sample)
values = clf.predict(X_test)
print(classification_report(values, y_test))
print((y_sample == 0).sum())
print((y_sample == 1).sum())

In [52]:
#TESTING SMOTE TECHNIQUE
X_train,X_test,y_train,y_test = train_test_split(X,y, test_size=0.3)
smote = SMOTE(ratio='minority')
x_sample, y_sample = smote.fit_sample(X_train, y_train)
clf = DecisionTreeClassifier()
clf.fit(x_sample,y_sample)
values = clf.predict(X_test)
print(classification_report(values, y_test))
print((y_sample == 0).sum())
print((y_sample == 1).sum())

TypeError: __init__() got an unexpected keyword argument 'ratio'

In [40]:
ros = RandomOverSampler()
rus = RandomUnderSampler()

def unbalanced_bagger(M, x, y):
    #M = Amount of base_learners, x = train set, y = class labels
    base_learners = []
    sampling_techniques = [ros, rus]
    for i in range(M):
        #boostrap replica
        rnd = np.random.choice(x.shape[0], size=x.shape[0])
        #x_bootstrap = x.iloc[rnd,:] #WHY?
        #y_bootstrap = y.iloc[rnd]
        #sample the replica with random sample technique
        technique = random.randint(0,len(sampling_techniques)-1)
        x_sample, y_sample = sampling_techniques[technique].fit_resample(x,y)
        #train
        clf = DecisionTreeClassifier()
        clf.fit(x_sample,y_sample)
        base_learners.append(clf)

    def predict(x):
        predictions = np.zeros([x.shape[0], len(base_learners)])
        for i in range(len(base_learners)):
            predictions[:,i] = base_learners[i].predict(x)
        #return np.sign(np.sum(predictions, axis=1))
        #return predictions.mode()
        return mode(predictions, axis=1)[0]
    return predict

In [41]:
X_train,X_test,y_train,y_test = train_test_split(X,y, test_size=0.3)
classifier = unbalanced_bagger(5, X_train, y_train)
#print(classifier(X_test))
#class_report = classifier(X_test)

In [42]:
value = classifier(X_test)

In [43]:
(value == 0).sum()

172133

In [44]:
(value == 1).sum()

6431

In [45]:
len(X_test)
print(classification_report(y_test, value))

              precision    recall  f1-score   support

           0       0.96      0.96      0.96    172014
           1       0.06      0.06      0.06      6550

    accuracy                           0.93    178564
   macro avg       0.51      0.51      0.51    178564
weighted avg       0.93      0.93      0.93    178564

